In [ ]:
from collections import defaultdict
from previous_code import dijktra_implementation, gradient_descent, 
                        neural_network,  

WALKING_MODE = 'Walking'
BIKING_MODE = 'Biking'
PUBLIC_MODE = 'Public Transport'
DRIVING_MODE = 'Driving'
DEFAULT_TRANSPORT_MODES = (WALKING_MODE, BIKING_MODE, 
                           PUBLIC_MODE, DRIVING_MODE)
EXAMPLE_GRAPHS_PER_MODE = {
    WALKING_MODE: HierarchicalGraph(),
    BIKING_MODE: HierarchicalGraph(),
    PUBLIC_MODE: HierarchicalGraph(),
    DRIVING_MODE: HierarchicalGraph()
}


class NavigationModel(object):
    """
    This Python class sketches an implementation of the 
    navigation model discussed in the paper. It is not
    meant to be fully working code; but rather, an example
    of how such code would look like
    """
    def __init__(self, transport_modes, h_graph_per_mode, training_data):
        self.transport_modes = transport_modes
        self.hierarchical_graphs = h_graph_per_mode
        self.ensembler_nn = neural_network()
        
    def find_routes(self, start, end):
        routes = []
        
        for mode in self.transport_modes:
            routes.extend(self.hierarchical_graphs[mode].find_routes(start, end))
            
        # sort by the prediction probabilities, in descending order
        routes.sort(key=self.ensembler_nn.predict, reverse=True)
        
        return routes
        
    def loss_function(self, predictions, true_probs):
        """
        Square error of the predicted probabilities
        """
        return sum([(pred - true_prob) ** 2 for pred, true_prob 
                   in zip(predictions, true_probs)])
    
    def error_function(self, training_data):
        """
        Average error over the entire data set
        """
        return 1.0 / len(training_data) * \
                sum([self.loss_function(
                        self.ensembler_nn.predict(data), 
                        true_probs)
                    for data, true_probs in training_data])
        
    def train_ensembler(self, training_data):
        """
        Use gradient descent with backpropogation through
        the neural network to train the model, using the
        loss function and error function defined above
        """
        self.ensembler_nn.train(training_data, 
                                self.loss_function,
                                self.error_function)
        

    
STATE_LEVEL = 'State'
REGION_LEVEL = 'Region'
CITY_LEVEL = 'City'

DEFAULT_LEVELS = (STATE_LEVEL, REGION_LEVEL, CITY_LEVEL)
EXAMPLE_GRAPHS_PER_LEVEL = {
    STATE: {'California': NavigationGraph()},
    REGION_LEVEL: {'SF Bay': NavigationGraph(),
                   'Central Valley': NavigationGraph(),
                   'Southern California': NavigationGraph()},
    CITY_LEVEL: {'Palo Alto': NavigationGraph(),
                 'Menlo Park': NavigationGraph(),
                 'Oakland': NavigationGraph(),
                 'San Francisco': NavigationGraph()}
}
DEFAULT_TOP_N = 3
    
class HierarchicalGraph(object):
    def __init__(self, levels=DEFAULT_LEVELS, 
                 graphs_per_level=EXAMPLE_GRAPHS_PER_LEVEL,
                 top_n=DEFAULT_TOP_N):
        self.levels = levels
        self.graphs_per_level = graphs_per_level
        self.top_n = top_n
        
    def find_routes(self, start, end):
        super_level = None
        results = defaultdict(list)
        
        for level in self.levels:
            # Starting and ending in the same place, ignore
            if start[level] == end[level]:
                super_level = level
                continue
            
            # Asked to route on a level not implemented 
            # (i.e. between countries) - abort
            if super_level is None:
                raise ValueError('Cannot route on this level')
            
            # If we have no information stored about the previous level
            # We compute routes from the raw start / end
            if super_level not in results:
                super_graph = self.graphs_per_level[super_level]
                routes = super_graph.dijkstra(start[level], end[level])
                results[level] = routes
                
            # If we have routes from the previous level
            else:
                # For each route computed in the previous level
                super_level_routes = routes[super_level]
                super_graph = self.graphs_per_level[super_level]
                for super_level_route in super_level_routes:
                    # Compute routes from the start to the start of the super_level route
                    start_routes = super_graph.dijkstra(start[level],
                                                        super_level_route.start)
                    # Compute routes from the end of the super_level route to the end
                    end_routes = super_graph.dijkstra(super_level_route.end,
                                                      end[level])
                    
                    # Assume such a function exists to combine all routes
                    route_combinations = concatanate_routes(start_routes, 
                                                           super_level_routes,
                                                           en_routes)
                    # Add in the new results
                    results[level].extend(route_combinations)
            
            super_level = level
            
        # return the first n results in some sorted order
        return sorted(results[level])[:self.top_n]

class NavigationGraph(object)
    """
    A Python class placeholder for a graph with the 
    specifications described and demanded by the model
    """
    def __init__(self, graph):
        self.graph = graph
        
    def dijkstra(self, start, end, num_results):
        if start not in self.graph:
            raise ValueError('Start value {s} not in graph'.format(s=start))
            
        if end not in self.graph:
            raise ValueError('End value {e} not in graph'.format(e=end))
            
        # insert implementation of Dijkstra here
        # since I already implemented it in the graph theory assignment
        # I will not repeat the implementation here
        
        return results